In [7]:
import pickle
import os
import sys
sys.path.append("../")
from od.social.group import SocialGroup
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS
from od.env.station import BS_PRESET

In [8]:
print(os.getcwd())
interest_config = InterestConfig(True,ResourceAllocatorType.NOMA_APR,False,0.6,666)
statistic = None
print(interest_config)
with open("../data/TVAL/{}/object.pickle".format(interest_config.folder()), "rb") as file:
    statistic = pickle.load(file)
statistic.nft_traffic = {}

/home/OD/repos/ProjectSumo/ipynb
yQoS-nRSU-NOMA_APR-D0.6-S666


In [9]:
class GV:
    NET_STATION_CONTROLLER = [
            bs_config for bs_config in BS_PRESET.values()
             if ((bs_config["type"] == BaseStationType.UMA)  or 
            (interest_config.req_rsu and bs_config["type"] == BaseStationType.UMI))
    ]
class STATISTIC:
    def Log(data):
        print(data)

In [10]:
GV.NET_STATION_CONTROLLER

[{'color': (0, 0, 0, 255),
  'pos': (160, 100),
  'img': '/home/OD/repos/ProjectSumo/ipynb/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMA: 1>},
 {'color': (0, 0, 0, 255),
  'pos': (450, 330),
  'img': '/home/OD/repos/ProjectSumo/ipynb/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMA: 1>}]

In [11]:
print(len(statistic.sg_header[SocialGroup.RCWS].keys()))
print(len(statistic.sg_header[SocialGroup.CRASH].keys()))

0
16216


In [14]:

for sg in statistic.social_group:
    ot_count = 0 
    for header, record in statistic.sg_header[sg].items():
        if record.is_src_ot > 0:
            ot_count +=1
    print("{}: {} => {}".format(sg,ot_count, ot_count / max(len(statistic.sg_header[sg]),1)))
            

CRASH: 0 => 0.0
RCWS: 0 => 0.0
DYN_SG_0: 0 => 0.0
DYN_SG_1: 0 => 0.0
DYN_SG_2: 0 => 0.0
DYN_SG_3: 0 => 0.0
DYN_SG_4: 1537 => 0.7805992889791773
DYN_SG_5: 0 => 0.0
DYN_SG_6: 482 => 0.4963954685890834
DYN_SG_7: 91 => 0.07647058823529412
DYN_SG_8: 366 => 0.7484662576687117
DYN_SG_9: 0 => 0.0


In [121]:

class Helper:
    def __init__(self,header,record,bs_index):
        self.name = header
        self.bits = record.bits
        self.time_gen = round(record.at,4)
        self.time_serv = round(record.time_bs_serv[bs_index],4)
        self.time_len = round(self.time_serv - self.time_gen,4)
    def __str__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)
    def __repr__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)


In [122]:
target_header = "flow_0.10-249"
header_in_sg_list = [sg.fname for sg in statistic.social_group   if(target_header in statistic.sg_header[sg])]
print("In {}".format(header_in_sg_list))
print("C?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL)))
print("G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.GENERAL)))
print("C2G?:{}".format(target_header in statistic.ExtractNetworkTraffic(NetFlowType.C2G)))

In ['DYN_SG_2']
C?:False
G?:True
C2G?:False


In [123]:
for sg in statistic.social_group:
    if target_header in statistic.sg_header[sg]:
        header = statistic.sg_header[sg][target_header]
print("bs ot:{}".format(header.is_bs_ot))
print("src ot:{}".format(header.is_src_ot))
print("time bs drop:{}".format(header.time_bs_drop))
print("time bs serv:{}".format(header.time_bs_serv))
print("time bs tx:{}".format(header.time_bs_tx))
print("time bs txq:{}".format(header.time_bs_txq))
print("time bs trip:{}".format(header.time_veh_trip))

bs ot:[False, False, False, False, False, False, False]
src ot:False
time bs drop:[-1, -1, -1, -1, -1, -1, -1]
time bs serv:[187.53, 187.53, -1, -1, -1, -1, -1]
time bs tx:[[[187.53, 187.531]], [[187.53, 187.531]], [], [], [], [], []]
time bs txq:[[[187.53, 187.53]], [[187.53, 187.53]], [], [], [], [], []]
time bs trip:{'flow_0.12': 0.0020000000000095497, 'flow_0.7': 0.0020000000000095497, 'flow_0.9': 0.0020000000000095497, 'flow_1.2': 0.0020000000000095497, 'flow_1.4': 0.0020000000000095497, 'flow_2.5': 0.0020000000000095497, 'flow_2.8': 0.0020000000000095497}


In [124]:
header.time_bs_txq

[[[187.53, 187.53]], [[187.53, 187.53]], [], [], [], [], []]

In [125]:
bs_index = 1

In [126]:
bs_serv_list = []
for header, record in statistic.ExtractNetworkTraffic(NetFlowType.GENERAL).items():
    if(record.time_bs_serv[bs_index] > 0):
        bs_serv_list.append(Helper(header,record,bs_index))
bs_serv_list.sort(key=lambda x: (x.time_gen*1000)*10000000 + (x.time_len*1000) )

In [127]:
bs_serv_list_crit = []
for header, record in statistic.ExtractNetworkTraffic(NetFlowType.CRITICAL).items():
    if(record.time_bs_serv[bs_index] > 0):
        bs_serv_list_crit.append(Helper(header,record,bs_index))
bs_serv_list_crit.sort(key=lambda x: (x.time_gen*1000)*10000000 + (x.time_len*1000))

In [128]:
for helper in bs_serv_list_crit:
    print(str(helper))

flow_1.2-64[ at:186.0 ,to:186.001, dur:0.001, size:8160]
flow_2.11-65[ at:186.001 ,to:186.002, dur:0.001, size:3040]
flow_2.14-65[ at:186.001 ,to:186.002, dur:0.001, size:5616]
flow_2.15-64[ at:186.001 ,to:186.002, dur:0.001, size:2960]
flow_2.16-64[ at:186.001 ,to:186.002, dur:0.001, size:2640]
flow_2.5-65[ at:186.001 ,to:186.002, dur:0.001, size:4368]
flow_0.11-64[ at:186.002 ,to:186.003, dur:0.001, size:4760]
flow_0.6-64[ at:186.002 ,to:186.003, dur:0.001, size:3272]
flow_2.8-64[ at:186.002 ,to:186.003, dur:0.001, size:6200]
flow_2.9-65[ at:186.002 ,to:186.003, dur:0.001, size:4176]
flow_0.13-64[ at:186.002 ,to:186.004, dur:0.002, size:8032]
flow_0.10-64[ at:186.003 ,to:186.004, dur:0.001, size:8632]
flow_0.7-65[ at:186.003 ,to:186.004, dur:0.001, size:5752]
flow_0.9-65[ at:186.003 ,to:186.004, dur:0.001, size:4808]
flow_0.12-65[ at:186.004 ,to:186.005, dur:0.001, size:8488]
flow_0.14-65[ at:186.004 ,to:186.005, dur:0.001, size:6544]
flow_0.4-65[ at:186.004 ,to:186.005, dur:0.001, s

In [129]:
for helper in bs_serv_list:
    print(str(helper))

flow_0.8-64[ at:186.0 ,to:186.001, dur:0.001, size:3560]
flow_2.11-64[ at:186.0 ,to:186.001, dur:0.001, size:1888]
flow_0.7-64[ at:186.001 ,to:186.002, dur:0.001, size:13920]
flow_2.15-65[ at:186.001 ,to:186.002, dur:0.001, size:3728]
flow_2.13-64[ at:186.001 ,to:186.002, dur:0.001, size:2744]
flow_1.4-65[ at:186.002 ,to:186.003, dur:0.001, size:9368]
flow_2.8-65[ at:186.002 ,to:186.003, dur:0.001, size:9024]
flow_0.13-65[ at:186.002 ,to:186.003, dur:0.001, size:7608]
flow_0.14-64[ at:186.002 ,to:186.003, dur:0.001, size:808]
flow_0.6-65[ at:186.002 ,to:186.003, dur:0.001, size:11728]
flow_2.16-65[ at:186.002 ,to:186.003, dur:0.001, size:10464]
flow_0.11-65[ at:186.003 ,to:186.004, dur:0.001, size:12232]
flow_0.10-65[ at:186.004 ,to:186.005, dur:0.001, size:11936]
flow_1.2-65[ at:186.005 ,to:186.006, dur:0.001, size:14816]
flow_13.0-65[ at:186.006 ,to:186.007, dur:0.001, size:2832]
flow_2.10-65[ at:186.009 ,to:186.01, dur:0.001, size:4104]
flow_2.12-65[ at:186.01 ,to:186.011, dur:0.001

In [130]:
time_gap = {}
for helper in bs_serv_list_crit:
    at = str(helper.time_gen)
    to = str(helper.time_serv)
    if(to not in  time_gap):
        time_gap[to] = {"need": 0, "give": 0}
    if(at not in time_gap):
        time_gap[at] = {"need": 0, "give": 0}
    time_gap[to]["give"]+=helper.bits
    time_gap[at]["need"]+=helper.bits
keys = list(time_gap.keys())
keys.sort()
for key in keys:
    print("{}:{}".format(key,str(time_gap[key])))

186.0:{'need': 8160, 'give': 0}
186.001:{'need': 18624, 'give': 8160}
186.002:{'need': 26440, 'give': 18624}
186.003:{'need': 19192, 'give': 18408}
186.004:{'need': 22184, 'give': 27224}
186.005:{'need': 17808, 'give': 22184}
186.006:{'need': 0, 'give': 17808}
186.016:{'need': 7160, 'give': 0}
186.017:{'need': 20016, 'give': 7160}
186.018:{'need': 15608, 'give': 20016}
186.019:{'need': 41064, 'give': 15608}
186.02:{'need': 3920, 'give': 26560}
186.021:{'need': 30352, 'give': 18424}
186.022:{'need': 15792, 'give': 30352}
186.023:{'need': 0, 'give': 15792}
186.032:{'need': 11328, 'give': 0}
186.033:{'need': 0, 'give': 11328}
186.034:{'need': 31520, 'give': 0}
186.035:{'need': 8616, 'give': 31520}
186.036:{'need': 26344, 'give': 8616}
186.037:{'need': 22344, 'give': 26344}
186.038:{'need': 16840, 'give': 15512}
186.039:{'need': 9952, 'give': 23672}
186.04:{'need': 0, 'give': 9952}
186.048:{'need': 5416, 'give': 0}
186.049:{'need': 4128, 'give': 5416}
186.05:{'need': 4608, 'give': 4128}
18

In [131]:
time_gap = {}
for helper in bs_serv_list:
    at = str(helper.time_gen)
    to = str(helper.time_serv)
    if(to not in  time_gap):
        time_gap[to] = {"need": 0, "give": 0}
    if(at not in time_gap):
        time_gap[at] = {"need": 0, "give": 0}
    time_gap[to]["give"]+=helper.bits
    time_gap[at]["need"]+=helper.bits
keys = list(time_gap.keys())
keys.sort()
for key in keys:
    print("{}:{}".format(key,str(time_gap[key])))

186.0:{'need': 5448, 'give': 0}
186.001:{'need': 20392, 'give': 5448}
186.002:{'need': 49000, 'give': 20392}
186.003:{'need': 12232, 'give': 49000}
186.004:{'need': 11936, 'give': 12232}
186.005:{'need': 14816, 'give': 11936}
186.006:{'need': 2832, 'give': 14816}
186.007:{'need': 0, 'give': 2832}
186.009:{'need': 4104, 'give': 0}
186.01:{'need': 11440, 'give': 4104}
186.011:{'need': 1320, 'give': 11440}
186.012:{'need': 0, 'give': 1320}
186.013:{'need': 14184, 'give': 0}
186.014:{'need': 16184, 'give': 14184}
186.015:{'need': 14336, 'give': 16184}
186.016:{'need': 34448, 'give': 14336}
186.017:{'need': 25320, 'give': 34448}
186.018:{'need': 16496, 'give': 25320}
186.019:{'need': 29624, 'give': 16496}
186.02:{'need': 0, 'give': 29624}
186.021:{'need': 23752, 'give': 0}
186.022:{'need': 15736, 'give': 23752}
186.023:{'need': 0, 'give': 15736}
186.024:{'need': 3472, 'give': 0}
186.025:{'need': 0, 'give': 3472}
186.026:{'need': 11512, 'give': 0}
186.027:{'need': 6256, 'give': 11512}
186.02

In [114]:
list(time_gap.keys()).sort()